In [ ]:
import rdflib
from rdflib import Namespace , Literal , URIRef
from rdflib.namespace import RDF , RDFS
import SPARQLWrapper
from SPARQLWrapper import SPARQLWrapper, JSON
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

# get the endpoint API
#wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
artchives_endpoint = "http://artchives.fondazionezeri.unibo.it/sparql"

# bind the uncommon namespaces
wd = Namespace("http://www.wikidata.org/entity/") # remember that a prefix matches a URI until the last slash (or hashtag #)
wdt = Namespace("http://www.wikidata.org/prop/direct/")
art = Namespace("https://w3id.org/artchives/")

# create an empty Graph
g = rdflib.ConjunctiveGraph()

# parse a local RDF file by specifying the format
result = g.parse("artchives.nq", format='nquads')

query_result = '''
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wdp: <http://www.wikidata.org/wiki/Property:>
PREFIX art: <https://w3id.org/artchives/>
SELECT DISTINCT ?coll ?artHistorian ?nameHistorian ?nameColl 
WHERE {
 ?coll wdp:P170 ?artHistorian .
 ?artHistorian rdfs:label ?nameHistorian .
 ?coll rdfs:label ?nameColl .
}
'''
query_2 = ''' 
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wdp: <http://www.wikidata.org/wiki/Property:>
PREFIX art: <https://w3id.org/artchives/>
SELECT DISTINCT ?period ?periodLabel ?collEarly ?collLate ?collLabel
WHERE {
?coll wdt:P19  ?collPlace ;
  wdt:P1319  ?collEarly ;
  wdt:P1326  ?collLate ;
  art:hasSubjectPeriod ?period .
 ?collPlace rdfs:label ?collLabel.
 ?period rdfs:label ?periodLabel .
 }
'''

sparql_wd = SPARQLWrapper(artchives_endpoint)
# set the query
sparql_wd.setQuery(query_result)
# set the returned format
sparql_wd.setReturnFormat(JSON)
# get the results
results = sparql_wd.query().convert() 
for result in results["results"]["bindings"]:
    print(result)

import pandas as pd
import json
import pprint

with open("data_query_collPeriods.json", "w") as write_file:
    json.dump(results, write_file)

with open("data_query_collPeriods.json", "r") as read_file:
    data = json.load(read_file)
    pprint.pprint(data)